In [5]:
import json
from datetime import date
from medcat.cat import CAT
from medcat.vocab import Vocab
from medcat.cdb import CDB
from tokenizers import ByteLevelBPETokenizer
import pandas as pd
from medcat.meta_cat import MetaCAT
import numpy as np


In [ ]:
data_path = '/data/mct-exports/<mct_export>.json' # from where to get the data, mctrainer export


# Meta paths to base
base_dir_meta_models = 'models/meta_models/'
vocab_file = ''
merges_file = ''
embeddings_file = ''

# will be used to date the trained model
today = str(date.today())
today = today.replace("-","")
save_dir = base_dir_meta_models + today+'_meta_models/' # Where to save the meta_models

In [ ]:
tokenizer = ByteLevelBPETokenizer(vocab_file=vocab_file,
                                merges_file=merges_file,
                                lowercase=True)

embeddings = np.load(open(embeddings_file, 'rb'))

In [ ]:
NAMES = ['Experiencer', 'Presence', 'Time'] # These Meta_annotation tasks should correspond to the ones labelled in the mcttrainer export
device = 'cuda' # or 'cpu' if you do not have GPU support

## Train Meta-Annotations

In [ ]:
results = {}
for NAME in NAMES:
    mc = MetaCAT(tokenizer, embeddings, cntx_left=20, cntx_right=5, save_dir=save_dir + NAME, device=device)
    result = mc.train(data_path, NAME, test_size=0.1, nepochs=40, lowercase=True, ignore_cpos=False, batch_size=1000,
        model_config={'num_layers':2}, lr=0.001, replace_center='disease')
    mc.save(full_save=True)

    result['names'] = mc.category_values
    results[NAME] = result


In [ ]:
# Overall results

json.dump(results, open(save_dir+'results.json', 'w'))